# 기본 CNN 구현

In [8]:
import sys, os
sys.path.append(os.pardir)  # 부모 디렉터리의 파일을 가져올 수 있도록 설정
import numpy as np
import pickle
from collections import OrderedDict
from common.layers import *
from common.gradient import numerical_gradient

class SimpleConvNet:
    '''
    input_dim - 입력 데이터(채널 수, 높이, 너비)
    conv_param - 합성곱 계층의 하이퍼파라미터(딕셔너리)
    -- filter_num = 필터수
    -- filter_size = 필터 크기
    -- stride = 스트라이드
    -- pad = 패딩
    -- hidden_size = 은닉층(완전연결)의 뉴런 수
    -- output_size = 출력층(완전연결)의 뉴런 수
    -- weight_init_std = 초기화 때의 가중치 표준편차
    '''
    
    def __init__(self, input_dim=(1, 28, 28),
                conv_param = {'filter_num}':30, 'filter_size':5, 'pad':0, 'stride':1},
                hidden_size=100, output_size=10, weight_init_std=0.01):
        filter_num = conv_param['filter_num']
        filter_size = conv_param['filter_size']
        filter_pad = conv_param['pad']
        filter_stride = conv_param['stride']
        input_size = input_dim[1]
        
        # 합성곱 계층의 출력 크기
        conv_output_size = (input_size - filter_size + 2*filter_pad) / filter_stride + 1
        
        # 풀링 계층의 출력 크기
        pool_output_size = int(filter_num * (conv_output_size / 2) * (conv_output_size / 2))
        
        # 가중치 초기화
        self.params = {}
        self.params['W1'] = weight_init_std * \
                            np.random.randn(filter_num, input_dim[0], filter_size, filter_size)
        self.params['b1'] = np.zeros(filter_num)
        self.params['W2'] = weight_init_std * \
                            np.random.randn(pool_output_size, hidden_size)
        self.params['b2'] = np.zeros(hidden_size)
        self.params['W3'] = weight_init_std * \
                            np.random.randn(hidden_size, output_size)
        self.params['b3'] = np.zeros(output_size)
        
        # 계층 생성
        self.layers = OrderDict()
        self.layers['Conv'] = Convolution(self.params['W1'], self.params['b1'], conv_param['stride'], conv_param['pad'])
        self.layers['Relu1'] = Relu()
        self.layers['Pool1'] = Pooling(pool_h=2, pool_w=2, stride=2)
        self.layers['Affine1'] = Affine(self.parmas['W2'], self.params['b2'])
        self.layers['Relu2'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W3'], self.params['b3'])
        
        self.last_layer = SoftmaxWithLoss()
        
    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)
        
        return x
    
    def loss(self, x, t):
        y = self.predict(x)
        
        return self.last_layer.forward(y, t)
    
    def gradient(self, x, t):
        
        # forward
        self.loss(x, t)

        # backward
        dout = 1
        dout = self.last_layer.backward(dout)

        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)

        # 결과 저장
        grads = {}
        grads['W1'], grads['b1'] = self.layers['Conv1'].dW, self.layers['Conv1'].db
        grads['W2'], grads['b2'] = self.layers['Affine1'].dW, self.layers['Affine1'].db
        grads['W3'], grads['b3'] = self.layers['Affine2'].dW, self.layers['Affine2'].db

        return grads
    
    def accuracy(self, x, t, batch_size=100):
        if t.ndim != 1 : t = np.argmax(t, axis=1)
        
        acc = 0.0
        
        for i in range(int(x.shape[0] / batch_size)):
            tx = x[i*batch_size:(i+1)*batch_size]
            tt = t[i*batch_size:(i+1)*batch_size]
            y = self.predict(tx)
            y = np.argmax(y, axis=1)
            acc += np.sum(y == tt) 
        
        return acc / x.shape[0]

In [9]:
import matplotlib.pyplot as plt
from dataset.mnist import load_mnist
from simple_convnet import SimpleConvNet
from common.trainer import Trainer

# 데이터 읽기
(x_train, t_train), (x_test, t_test) = load_mnist(flatten=False)

# 시간이 오래 걸릴 경우 데이터를 줄인다.
#x_train, t_train = x_train[:5000], t_train[:5000]
#x_test, t_test = x_test[:1000], t_test[:1000]

max_epochs = 20

network = SimpleConvNet(input_dim=(1,28,28), 
                        conv_param = {'filter_num': 30, 'filter_size': 5, 'pad': 0, 'stride': 1},
                        hidden_size=100, output_size=10, weight_init_std=0.01)
                        
trainer = Trainer(network, x_train, t_train, x_test, t_test,
                  epochs=max_epochs, mini_batch_size=100,
                  optimizer='Adam', optimizer_param={'lr': 0.001},
                  evaluate_sample_num_per_epoch=1000)
trainer.train()

# 매개변수 보존
network.save_params("params.pkl")
print("Saved Network Parameters!")

# 그래프 그리기
markers = {'train': 'o', 'test': 's'}
x = np.arange(max_epochs)
plt.plot(x, trainer.train_acc_list, marker='o', label='train', markevery=2)
plt.plot(x, trainer.test_acc_list, marker='s', label='test', markevery=2)
plt.xlabel("epochs")
plt.ylabel("accuracy")
plt.ylim(0, 1.0)
plt.legend(loc='lower right')
plt.show()


train loss:2.29927186711489
=== epoch:1, train acc:0.181, test acc:0.193 ===
train loss:2.2986847965333554
train loss:2.295721698092865
train loss:2.2898670914782913
train loss:2.2835651093639915
train loss:2.271047211014419
train loss:2.252790354297983
train loss:2.236141653621637
train loss:2.23260546515645
train loss:2.2182793429729815
train loss:2.1822454780037037
train loss:2.1623701657794476
train loss:2.1559971138372687
train loss:2.0857599182445963
train loss:1.9842707678268274
train loss:1.928239576013476
train loss:1.9223302773922282
train loss:1.8340434086910862
train loss:1.7520853802082212
train loss:1.8338031893242925
train loss:1.6859393505097233
train loss:1.6727622875882444
train loss:1.5661912961794846
train loss:1.4519782085610555
train loss:1.4360577728007455
train loss:1.3223392982443423
train loss:1.2783107949890968
train loss:1.2538239636284736
train loss:1.1440150104310327
train loss:0.9767240515195356
train loss:0.8644166803971823
train loss:1.0231945697843683


train loss:0.19534724302609227
train loss:0.23690151065304252
train loss:0.27585486295547207
train loss:0.15218082961402485
train loss:0.3188717014568369
train loss:0.26914016982065847
train loss:0.2748806633742198
train loss:0.16138886303325306
train loss:0.21417967833678675
train loss:0.1711067362220046
train loss:0.22876714348539337
train loss:0.1971054914796428
train loss:0.18514164840337785
train loss:0.26783614014611207
train loss:0.2092698605604314
train loss:0.2630528445809724
train loss:0.19825488383614281
train loss:0.22387669016693923
train loss:0.22071184472080788
train loss:0.2405553135169628
train loss:0.44513564879001777
train loss:0.16000509458556578
train loss:0.3676282634185858
train loss:0.15771629601998155
train loss:0.2228269737568646
train loss:0.21791241357107388
train loss:0.16475135440801403
train loss:0.2478103155208581
train loss:0.2879880607855321
train loss:0.30284179144952766
train loss:0.2363705536915644
train loss:0.20658896929199153
train loss:0.3044936

train loss:0.26005343803578135
train loss:0.22778466278308968
train loss:0.1779623562671724
train loss:0.20420175161617574
train loss:0.1491695129363074
train loss:0.2547354228511863
train loss:0.11785908038951609
train loss:0.10638069892472689
train loss:0.13278636639399036
train loss:0.08810705140251616
train loss:0.05033584775220915
train loss:0.08208554975586095
train loss:0.17801038193841684
train loss:0.14069061964213725
train loss:0.09512521861355668
train loss:0.14087062076266413
train loss:0.14350368878492487
train loss:0.17387256870230153
train loss:0.12343026473022142
train loss:0.1251315722647959
train loss:0.09283702281337254
train loss:0.1152312288063977
train loss:0.127099661925094
train loss:0.21879391509671456
train loss:0.058527245276980694
train loss:0.12558390806374753
train loss:0.18249421751360922
train loss:0.2332199036051571
train loss:0.07058142705359019
train loss:0.19558738355523225
train loss:0.09097312174782561
train loss:0.14090523142553116
train loss:0.15

train loss:0.0780871427614414
train loss:0.1717873668770405
train loss:0.05804914280178726
train loss:0.11169297390605132
train loss:0.08923500563921612
train loss:0.13425260540112716
train loss:0.15770000374104223
train loss:0.07785801619044573
train loss:0.10194161144164675
train loss:0.07098300809424907
train loss:0.03800165747170095
train loss:0.1400624479723006
train loss:0.3461621262040899
train loss:0.09116396461148567
train loss:0.167761032161172
train loss:0.07159484503927929
train loss:0.06769714511167395
train loss:0.1762686599980824
train loss:0.06163140048384801
train loss:0.07784337347755813
train loss:0.10731509721243189
train loss:0.08794748432639626
train loss:0.15013568319059442
train loss:0.13054913373367039
train loss:0.03270725275381548
train loss:0.06054885652164766
train loss:0.1112361265856507
train loss:0.06962339803508258
train loss:0.11355348747066335
train loss:0.04445310976193692
train loss:0.09756324684147062
train loss:0.08597327015792708
train loss:0.073

train loss:0.09616397584551409
train loss:0.02872019729307478
train loss:0.18226201531148922
train loss:0.05624828379230232
train loss:0.14276888065233168
train loss:0.0951005874691871
train loss:0.09916740631587788
train loss:0.03933248186068903
train loss:0.070287756623908
train loss:0.05903779297818212
train loss:0.03803634097196289
train loss:0.08565236236198563
train loss:0.030410766969962068
train loss:0.07610640762890426
train loss:0.09121514331100343
train loss:0.12362040032665661
train loss:0.12110744372570931
train loss:0.03362420493601092
train loss:0.027175861237778112
train loss:0.05985083869030612
train loss:0.07364410360075538
train loss:0.01438072166383376
train loss:0.05992541443292351
train loss:0.09871525358302179
train loss:0.08021484709971849
train loss:0.08722074868585387
train loss:0.06502851912250641
train loss:0.04394657729012375
train loss:0.06500412673041986
train loss:0.06553515263202149
train loss:0.12061665271270168
train loss:0.05061293422188802
train los

...

train loss:0.0007659247427367412
train loss:0.0009179005642632118
train loss:0.0039019951630735543
train loss:0.0015091299760054877
train loss:0.004320235700581162
train loss:0.0029774739999986572
train loss:0.00510762120887616
train loss:0.0032030124869461907
train loss:0.0014149108370680862
train loss:0.002889540800970688
train loss:0.0013112161128612805
train loss:0.0005813123474053661
train loss:0.0036238060939446474
train loss:0.0012433522703139805
train loss:0.0002153792751802687
train loss:0.005119532636345232
train loss:0.0023213175488065647
train loss:0.0018509864172029052
train loss:0.004832631517807171
train loss:0.006799861704685455
train loss:0.0007204649470142935
train loss:0.0010082361387744661
train loss:0.000299585284789989
train loss:0.0009429429767365812
train loss:0.007774141108059118
train loss:0.0008058740995950145
train loss:0.0026360396639796323
train loss:0.0008796423376746921
train loss:0.0011058921357279359
train loss:0.0017440421915742486
train loss:0.027211

train loss:0.0023753254104871583
train loss:0.00033705733212288744
train loss:0.000310338167524693
train loss:0.00023059297555287443
train loss:0.003974102504005702
train loss:0.010771331786929593
train loss:0.0017102867092968269
train loss:0.0004214718069301654
train loss:0.029085968118644834
train loss:0.0036765954106183856
train loss:0.00343285154978277
train loss:0.002990338890104073
train loss:0.0006419252093398608
train loss:0.0002424176891222694
train loss:0.0009019824144247139
train loss:0.00039163462969758707
train loss:0.0032928287516861077
train loss:0.004524717834317889
train loss:0.001119232978047914
train loss:0.013836187802317825
train loss:0.006774033179805666
train loss:0.0007980623145627628
train loss:0.006813539608370031
train loss:0.002288228580984115
train loss:0.008487141120330109
train loss:0.0007005391701738112
train loss:0.008923229431598092
train loss:0.003998970605172107
train loss:0.019027805475802743
train loss:0.0038592720535793918
train loss:0.00106407378

train loss:0.005865066314774737
train loss:0.001458159680010628
train loss:0.004948292664711606
train loss:0.005352261121060725
train loss:0.006479881901392055
train loss:0.0012177848797068264
train loss:0.0014036005654413355
train loss:0.0004142871494845136
train loss:0.012879994341662891
train loss:0.001600025409065577
train loss:0.0008910545182560184
train loss:0.0012532869045516954
train loss:0.0007078044899068415
train loss:0.002990767250581624
train loss:0.0016111829581915483
train loss:0.0007880263577899753
train loss:0.026110065280789782
train loss:0.03310098477366158
train loss:0.0002814195528877845
train loss:0.016863334227696328
train loss:0.0005497342998170496
train loss:0.001701898101396798
train loss:0.003783687739403739
train loss:0.0019147217921252646
train loss:0.007095553265227325
train loss:0.0008502743085792812
train loss:0.00039677901749677946
train loss:0.0009320385396173424
train loss:0.00385145385225241
train loss:0.0005208886956286191
train loss:0.0251794561344

train loss:0.007092685359927267
train loss:0.0016662292949924675
train loss:0.001129401284409771
train loss:0.00016241591823498914
train loss:0.00010757788094093109
train loss:0.006834605407405237
train loss:0.0025295218257296082
train loss:0.003069640757084858
train loss:0.0029650282382219133
train loss:0.0026448726803207034
train loss:0.0032170230849519095
train loss:0.0029900152794243872
train loss:0.00936582146823177
train loss:0.00016158586982049932
train loss:0.00046598591243530773
train loss:0.0014925888742643126
train loss:0.00015157468662813686
train loss:0.0023635559061388057
train loss:0.0006869790501057515
train loss:0.0048934484291231544
train loss:0.004144713412742919
train loss:0.013795484582228783
train loss:0.005732626053364334
train loss:0.0007425024276114986
train loss:0.013707931402217799
train loss:0.004911638199451866
train loss:0.0015104209747997342
train loss:0.0005277050276433818
train loss:0.0006617264852518713
train loss:0.009126611631558985
train loss:0.0043

KeyboardInterrupt: 